<a href="https://colab.research.google.com/github/minicks/BigI/blob/master/titanic4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split



데이터 불러오기

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/test.csv')
combine = [train_df, test_df]

# 전처리

## train 전 처리

전처리('Embarked') 문자를 숫자값으로

In [0]:
for index in train_df[train_df['Embarked'] == 'C'].index:
  train_df.loc[index,'Embarked'] = 0

for index in train_df[train_df['Embarked'] == 'Q'].index:
  train_df.loc[index,'Embarked'] = 1

for index in train_df[train_df['Embarked'] == 'S'].index:
  train_df.loc[index,'Embarked'] = 2

for index in train_df[train_df['Embarked'].isnull()].index:
  train_df.loc[index,'Embarked'] = 0

전처리('Sex') 문자를 숫자값으로

In [0]:
for index in train_df[train_df['Sex'] == 'male'].index:
  train_df.loc[index,'Sex'] = 0

for index in train_df[train_df['Sex'] == 'female'].index:
  train_df.loc[index,'Sex'] = 1

나이 변환

In [0]:
for index in train_df[train_df['Age'].isnull()].index:
  if (train_df.loc[index,'SibSp']==0) & (train_df.loc[index,'Parch']==0):
    train_df.loc[index,'Age'] = 32.0
  elif (train_df.loc[index,'SibSp']==0) & ((2>=train_df.loc[index,'Parch']) & (train_df.loc[index,'Parch'] > 0)):
    train_df.loc[index,'Age'] = 26.0
  elif (train_df.loc[index,'SibSp'] == 0) & (train_df.loc[index,'Parch'] > 2):
    train_df.loc[index,'Age'] = 32.0
  elif ((2 >= train_df.loc[index,'SibSp']) & (train_df.loc[index,'SibSp'] > 0)) & (train_df.loc[index,'Parch']==0):
    train_df.loc[index,'Age'] = 32.0
  elif ((2 >= train_df.loc[index,'SibSp']) & (train_df.loc[index,'SibSp'] > 0)) & ((2>=train_df.loc[index,'Parch']) &(train_df.loc[index,'Parch'] > 0)):
    train_df.loc[index,'Age'] = 23.0
  elif ((2>=train_df.loc[index,'SibSp']) & (train_df.loc[index,'SibSp'] > 0)) & (train_df.loc[index,'Parch'] > 2):
    train_df.loc[index,'Age'] = 40.0
  elif (train_df.loc[index,'SibSp']>2) & (train_df.loc[index,'Parch']==0):
    train_df.loc[index,'Age'] = 31.0
  elif (train_df.loc[index,'SibSp']>2) & ((2>=train_df.loc[index,'Parch']) &(train_df.loc[index,'Parch'] > 0)):
    train_df.loc[index,'Age'] = 8.0
  elif (train_df.loc[index,'SibSp']>2) & (train_df.loc[index,'Parch'] > 2):
    train_df.loc[index,'Age'] = 32.0


In [91]:
# 분포 확인
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.369942
2,"(32.252, 48.168]",0.404255
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [0]:
train_df = train_df.drop(['AgeBand'], axis=1)

In [90]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 5)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.854]",0.217877
1,"(7.854, 10.5]",0.201087
2,"(10.5, 21.679]",0.424419
3,"(21.679, 39.688]",0.444444
4,"(39.688, 512.329]",0.642045


In [0]:
train_df = train_df.drop(['FareBand'], axis=1)

원핫인코딩

In [64]:
train_df["Embarked_C"] = train_df["Embarked"] == "C"
train_df["Embarked_S"] = train_df["Embarked"] == "S"
train_df["Embarked_Q"] = train_df["Embarked"] == "Q"

def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data


dummy_columns = ["Sex", "Pclass", "Embarked"]
train_df = dummy_data(train_df, dummy_columns)

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## test 전처리

전처리('Embarked') 문자를 숫자값으로

In [0]:
for index in test_df[test_df['Embarked'] == 'C'].index:
  test_df.loc[index,'Embarked'] = 0

for index in test_df[test_df['Embarked'] == 'Q'].index:
  test_df.loc[index,'Embarked'] = 1

for index in test_df[test_df['Embarked'] == 'S'].index:
  test_df.loc[index,'Embarked'] = 2

for index in test_df[test_df['Embarked'].isnull()].index:
  test_df.loc[index,'Embarked'] = 0

전처리('Sex') 문자를 숫자값으로

In [0]:
for index in test_df[test_df['Sex'] == 'male'].index:
  test_df.loc[index,'Sex'] = 0

for index in test_df[test_df['Sex'] == 'female'].index:
  test_df.loc[index,'Sex'] = 1

나이 변환

In [0]:
for index in test_df[test_df['Age'].isnull()].index:
  if (test_df.loc[index,'SibSp']==0) & (test_df.loc[index,'Parch']==0):
    test_df.loc[index,'Age'] = 32.0
  if (test_df.loc[index,'SibSp']==0) & ((2>=test_df.loc[index,'Parch']) &(test_df.loc[index,'Parch'] > 0)):
    test_df.loc[index,'Age'] = 26.0
  if (test_df.loc[index,'SibSp']==0) & (test_df.loc[index,'Parch'] > 2):
    test_df.loc[index,'Age'] = 32.0
  if ((2>=test_df.loc[index,'SibSp']) &(test_df.loc[index,'SibSp'] > 0)) & (test_df.loc[index,'Parch']==0):
    test_df.loc[index,'Age'] = 32.0
  if ((2>=test_df.loc[index,'SibSp']) &(test_df.loc[index,'SibSp'] > 0)) & ((2>=test_df.loc[index,'Parch']) &(test_df.loc[index,'Parch'] > 0)):
    test_df.loc[index,'Age'] = 23.0
  if ((2>=test_df.loc[index,'SibSp']) &(test_df.loc[index,'SibSp'] > 0)) & (test_df.loc[index,'Parch'] > 2):
    test_df.loc[index,'Age'] = 40.0
  if (test_df.loc[index,'SibSp']>2) & (test_df.loc[index,'Parch']==0):
    test_df.loc[index,'Age'] = 31.0
  if (test_df.loc[index,'SibSp']>2) & ((2>=test_df.loc[index,'Parch']) &(test_df.loc[index,'Parch'] > 0)):
    test_df.loc[index,'Age'] = 8.0
  if (test_df.loc[index,'SibSp']>2) & (test_df.loc[index,'Parch'] > 2):
    test_df.loc[index,'Age'] = 32.0


In [0]:
combine = [train_df, test_df]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)


In [0]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16.336, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16.336) & (dataset['Age'] <= 32.252), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32.252) & (dataset['Age'] <= 48.168), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48.168) & (dataset['Age'] <= 64.084), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64.084, 'Age'] = 4

In [0]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.854, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.854) & (dataset['Fare'] <= 10.5), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 10.5) & (dataset['Fare'] <= 21.679), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 21.679) & (dataset['Fare'] <= 39.688), 'Fare']   = 3
    dataset.loc[ dataset['Fare'] > 39.688, 'Fare'] = 4

원핫인코딩

In [48]:
test_df["Embarked_C"] = test_df["Embarked"] == "C"
test_df["Embarked_S"] = test_df["Embarked"] == "S"
test_df["Embarked_Q"] = test_df["Embarked"] == "Q"

test_df = dummy_data(test_df, dummy_columns)

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


col 삭제

In [0]:
# Name,Ticket,Cabin col 삭제
df_1 = train_df.drop(columns=['Name','Ticket','Cabin'])

In [0]:
#train,valid 나누기 
train0,test0 = train_test_split(df_1,test_size=0.3, random_state=1)

In [0]:
# Name,Ticket,Cabin col 삭제
df_2 = test_df.drop(columns=['Name','Ticket','Cabin'])

None 값 채우기

In [0]:
train0=train0.fillna(0.0)
test0=test0.fillna(10.0)

In [0]:
df_2=df_2.fillna(20.0)

## 회귀 넣기 전 데이터 *처리*

In [0]:
Ytrain = train0["Survived"]
Xtrain = train0.drop("Survived",axis = 1)
Ytest = test0["Survived"]
Xtest = test0.drop("Survived",axis = 1)
Ydf_1 = df_1["Survived"]
Xdf_1 = df_1.drop("Survived",axis = 1)

In [0]:
Xtrain=Xtrain.astype('float32')
Ytrain=Ytrain.astype('float32')
Xtest=Xtest.astype('float32')
Ytest=Ytest.astype('float32')
Xdf_1=Xdf_1.astype('float32')
Ydf_1=Ydf_1.astype('float32')
df_2=df_2.astype('float32')


## 전처리 test

In [0]:
for index in train_df[train_df['Age'].isnull()].index:
  train_df=train_df.drop(index=index)

In [0]:
for index in test_df[test_df['Age'].isnull()].index:
  test_df=test_df.drop(index=index)

test

In [0]:
train_df[((2>=train_df['SibSp']) &(train_df['SibSp'] > 0)) & (train_df['Parch'] > 2)].describe()

In [0]:
train_df[((2>=train_df['SibSp']) &(train_df['SibSp'] > 0)) & ((2>=train_df['Parch']) &(train_df['Parch'] > 0))].describe()

In [0]:
train_df[((2>=train_df['SibSp']) &(train_df['SibSp'] > 0)) & (train_df['Parch'] == 0)].describe()

In [0]:
df_1.info()

In [0]:
df_2.info()

In [0]:
Xdf_1

In [0]:
Xtrain

In [0]:
for a in XXtrain.isnull():
  print(a)

# 회귀 분석

In [108]:
random_forest = RandomForestClassifier(n_estimators=1000)
random_forest.fit(Xtrain, Ytrain)
Ftrain = random_forest.predict(Xtrain)
Ydf_2 = random_forest.predict(df_2)
random_forest.score(Xtest,Ytest )
random_forest.score(Xtrain, Ytrain )
random_forest.score(df_2, Ydf_2 )
acc_random_forest1 = round(random_forest.score(Xtrain, Ftrain ) * 100, 2)
acc_random_forest2 = round(random_forest.score(Xtest,Ytest ) * 100, 2)
acc_random_forest3 = round(random_forest.score(df_2,Ydf_2 ) * 100, 2)
print('train : ' , acc_random_forest1)
print('valid : ' , acc_random_forest2)
print('test : ' ,acc_random_forest3)

train :  100.0
valid :  76.12
test :  100.0


In [109]:
random_forest = RandomForestClassifier(n_estimators=1000)
random_forest.fit(Xdf_1, Ydf_1)
Ftrain = random_forest.predict(Xtrain)
Ydf_2 = random_forest.predict(df_2)
random_forest.score(Xtest,Ytest )
random_forest.score(Xtrain, Ytrain )
random_forest.score(df_2, Ydf_2 )
acc_random_forest1 = round(random_forest.score(Xtrain, Ftrain ) * 100, 2)
acc_random_forest2 = round(random_forest.score(Xtest,Ytest ) * 100, 2)
acc_random_forest3 = round(random_forest.score(df_2,Ydf_2 ) * 100, 2)
print('train : ' , acc_random_forest1)
print('valid : ' , acc_random_forest2)
print('test : ' ,acc_random_forest3)

train :  100.0
valid :  100.0
test :  100.0


# 파일 저장

In [0]:
submission = pd.DataFrame({
        "PassengerId": df_2["PassengerId"],
        "Survived": Ydf_2
    })
submission.to_csv('Fare_5'+'titanic_SUBMISSION.csv', index=False)

# 정보 모으기

In [0]:
print(train_df.sort_values(by='Survived'))
print(train_df.columns.values)
print('======================================================')
print(test_df)
print(test_df.columns.values)
print('======================================================')
print(combine)
print('======================================================')

In [0]:
print(train_df.min())
print(train_df.max())
train_df.describe(include='all')

In [0]:
train_df.info()

In [0]:
for a in train_df.index[train_df['Age'].isnull()]:
  print(a)

In [0]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [0]:
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Cabin', alpha=.5, bins=20)
grid.add_legend();

In [0]:
for index in train_df[train_df['Age'].isnull()].index:
  print(train_df.loc[index,['Pclass','SibSp','Parch']])

In [0]:

for index in train_df[(train_df['Age'].isnull()) & (train_df['SibSp'] == 0) & (train_df['Parch'] == 0)].index:
  print(train_df.loc[index,['Pclass','SibSp','Parch','Fare','Embarked','Cabin']])

In [0]:
train_df.corr(method='pearson')

In [0]:
train_df.info()

In [0]:
train_df.corr(method='pearson')

In [0]:
train_df.describe()

In [0]:
for index in train_df[train_df['Pclass'] == 1].index:
  train_df.loc[index,'Pclass'] = 'a'

In [0]:
for index in train_df[train_df['Pclass'] == 3].index:
  print(train_df.loc[index,'Cabin'])

# 새 섹션

In [0]:
Ytrain = train0["Survived"]
Xtrain = train0.drop("Survived",axis = 1)
Xtest = test0.drop("Survived",axis = 1)

In [0]:
Xtrain=Xtrain.astype('float32')
Ytrain=Ytrain.astype('float32')
Xtest=Xtest.astype('float32')

In [0]:
for index in Xtrain[Xtrain.isnull()].index:
  print(Xtrain.loc[index])

In [0]:
Xtrain[Xtrain.isnull()]

In [0]:
Xtest.info()

In [0]:
Xtrain[Xtrain.isnull()].index

In [0]:
Ypred

In [0]:
df_2